# BB84 simulation

The BB84 Quantum Key Distribution protocol is a method which uses the properties of quantum mechanics to ensure secure communication. It is capable of detecting eavesdropping attempts while generating a shared encryption key for Alice and Bob.

Import the required libraries.

In [8]:
from qiskit import QuantumCircuit, execute, Aer
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np
from qiskit.providers.aer import QasmSimulator

 Initially, Alice randomly selects a state and her basis and stores them in two lists. She does this by using the numpy function 'randint', which generates a list of random binary digits (each digit is 0 or 1). We assume 0 generated by `randomint` serve as Hadamard basis and 1 as Computational basis. Similarly, Bob also sets his basis by randomly generating a list of binary bits.

In [3]:
num_of_qubits = 128

state_of_alice = np.random.randint(2, size=num_of_qubits)
basis_of_alice = np.random.randint(2, size=num_of_qubits)
basis_of_bob = np.random.randint(2, size=num_of_qubits)

print(f"State of Alice:\t {np.array2string(state_of_alice)}")
print(f"Basis of Alice:\t {np.array2string(basis_of_alice)}")
print(f"Basis of Bob:\t {np.array2string(basis_of_bob)}")

State of Alice:	 [1 1 1 0 1 1 0 1 1 1 0 1 0 1 1 0 0 0 0 1 1 0 1 1 1 0 1 1 0 0 1 0 1 1 1 1 1
 1 0 0 1 0 1 1 1 1 1 0 0 0 1 0 0 0 1 1 0 1 1 0 1 1 0 1 1 0 0 0 1 1 1 0 0 0
 0 1 1 0 1 1 0 1 1 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0 1 1 1 1 0 0 1 0 0 1
 0 1 0 1 1 1 0 0 1 0 1 1 0 0 0 0 0]
Basis of Alice:	 [0 0 0 0 1 1 1 0 0 0 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 0 0 0 1 0 0 0 1 1 0 1
 0 0 0 0 1 1 0 0 1 0 1 0 1 0 1 0 1 0 1 1 0 0 1 0 0 0 1 1 0 0 0 1 1 0 1 1 1
 1 1 0 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 1 0 1 1 1 1 0 1 0 0 0 1 0 0 0
 0 0 1 1 0 0 1 0 0 0 1 1 0 1 1 0 0]
Basis of Bob:	 [0 1 1 1 0 1 1 0 1 0 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 0 0 0 0 0 1 0 1
 0 0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 1 1 1 0 1 1 1 0 0 0 0 1 1 0 1 1 1 1 1 0 0
 0 0 1 1 0 1 1 1 0 0 0 0 1 1 0 0 0 1 1 0 1 0 0 1 0 1 0 1 1 0 1 1 1 1 1 0 0
 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0]


The `create_bb84_circuit` function creates the Quantum Circuit in line with the BB84 protocol. Alice's randomly decided states and bases are used to prepare quantum states, while Bob's basis is used to make a measurement in the correct basis.

In [4]:
def create_bb84_circuit(input_qstate, input_qbasis, measurement_qbasis):
    qubits_count = len(input_qstate)
    created_circuit = QuantumCircuit(qubits_count)

    # sender
    for i in range(len(input_qbasis)):
        if input_qstate[i] == 1:
            created_circuit.x(i)
        if input_qbasis[i] == 1:
            created_circuit.h(i)

    # Measurement operation
    for i in range(len(measurement_qbasis)):
        if measurement_qbasis[i] == 1:
            created_circuit.h(i)

    created_circuit.measure_all()

    return created_circuit

The 'execute' function is used to start a quantum simulation. The circuit's bits are reversed prior to execution for compatibility with the simulator. The execution result is obtained and the most frequent memory string is retrieved, which effectively represents a binary string.

In [5]:
created_circuit = create_bb84_circuit(state_of_alice, basis_of_alice, basis_of_bob)
generated_key = execute(created_circuit.reverse_bits(), backend=QasmSimulator(), shots=1).result().get_counts().most_frequent()
secure_key = ''
for i in range(num_of_qubits):
    if basis_of_alice[i] == basis_of_bob[i]:
         secure_key += str(generated_key[i])
print(f"Secure Key: {secure_key}")

Secure Key: 110110100000110111011011111011001011010110010110011001101010110000


Visualize the generated quantum circuit.

In [6]:
created_circuit.draw(output='mpl')

We calculate the success probability by determining the ratio of matching basis between Alice and Bob. The higher this success probability, the better the key distribution. Finally, it prints the secure key generated, the associated success probability, and the length of the unmatched key. This unmatched key length indicates errors or potential eavesdropping attempts in quantum channels.

In [7]:
from qiskit import execute
from qiskit.providers.aer import QasmSimulator

# Define the number of qubits
num_qubits = 128

# Alice generates her state and basis randomly
state_of_alice = np.random.randint(2, size=num_qubits)
basis_of_alice = np.random.randint(2, size=num_qubits)

# Bob also generates his basis in a random fashion
basis_of_bob = np.random.randint(2, size=num_qubits)

# Build the BB84 protocol quantum circuit
created_circuit = create_bb84_circuit(state_of_alice, basis_of_alice, basis_of_bob)
execution_results = execute(created_circuit.reverse_bits(), backend=QasmSimulator(), shots=1).result()
obtained_key = execution_results.get_counts().most_frequent()

# Create the secure encryption key
secure_key = ''
for i in range(num_qubits):
    if basis_of_alice[i] == basis_of_bob[i]:
         secure_key += str(obtained_key[i])

# Calculate the probability of successful encryption
unmatched_qubits = [i for i in range(num_qubits) if basis_of_alice[i] != basis_of_bob[i]]
success_prob = len(secure_key) / len(basis_of_alice)

print(f"Secure Key: {secure_key}")
print(f"Success Probability: {success_prob}")
print(f"Unmatched Key Length: {len(unmatched_qubits)}")

Secure Key: 11011111111011010100101000000011100000000110011111011110101100111
Success Probability: 0.5078125
Unmatched Key Length: 63
